<a href="https://colab.research.google.com/github/Takumi173/DifyApps/blob/main/%E8%96%AC%E5%89%A4%E5%B8%AB%E8%A9%A6%E9%A8%93%E3%82%92%E8%A7%A3%E3%81%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.mhlw.go.jp/content/001455149.pdf
!wget https://www.mhlw.go.jp/content/001455152.pdf
!wget https://www.mhlw.go.jp/content/001455159.pdf
!wget https://www.mhlw.go.jp/content/001455160.pdf
!wget https://www.mhlw.go.jp/content/001455161.pdf
!wget https://www.mhlw.go.jp/content/001455162.pdf

!pip install pymupdf pypdf

--2025-04-16 15:19:14--  https://www.mhlw.go.jp/content/001455149.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 23.65.0.57, 2600:1407:3c00:1a87::3194, 2600:1407:3c00:1a9d::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|23.65.0.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1485988 (1.4M) [application/pdf]
Saving to: ‘001455149.pdf’

001455149.pdf       100%[===================>]   1.42M  8.73MB/s    in 0.2s    

2025-04-16 15:19:14 (8.73 MB/s) - ‘001455149.pdf’ saved [1485988/1485988]

--2025-04-16 15:19:15--  https://www.mhlw.go.jp/content/001455152.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 23.65.0.57, 2600:1407:3c00:1a87::3194, 2600:1407:3c00:1a9d::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|23.65.0.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1992687 (1.9M) [application/pdf]
Saving to: ‘001455152.pdf’

001455152.pdf       100%[===================>]   1.90M  11.2MB/s    in 0.2s    

2025-04-16 15:19:15

In [2]:
from pypdf import PdfReader, PdfWriter

def split_pdf_pages(input_pdf_path, split_page):
    reader = PdfReader(input_pdf_path)
    writer = PdfWriter()

    output_filename = input_pdf_path.split(".")[0]

    start_index = 0
    end_index = len(reader.pages)


    # 前半ページを作成
    for i in range(start_index, split_page):
        writer.add_page(reader.pages[i])

    # 新しいPDFファイルとして保存
    with open(output_filename+"_1.pdf", "wb") as output_file:
        writer.write(output_file)


    writer = PdfWriter()

    # 後半ページを作成
    for i in range(split_page, end_index):
        writer.add_page(reader.pages[i])

    # 新しいPDFファイルとして保存
    with open(output_filename+"_2.pdf", "wb") as output_file:
        writer.write(output_file)


    print(f"'{output_filename}_1.pdf' と '{output_filename}_2.pdf' に分割しました 。")

split_pdf_pages("001455149.pdf", 20)
split_pdf_pages("001455152.pdf", 26)
split_pdf_pages("001455159.pdf", 17)
split_pdf_pages("001455160.pdf", 25)
split_pdf_pages("001455161.pdf", 21)
split_pdf_pages("001455162.pdf", 31)

'001455149_1.pdf' と '001455149_2.pdf' に分割しました 。
'001455152_1.pdf' と '001455152_2.pdf' に分割しました 。
'001455159_1.pdf' と '001455159_2.pdf' に分割しました 。
'001455160_1.pdf' と '001455160_2.pdf' に分割しました 。
'001455161_1.pdf' と '001455161_2.pdf' に分割しました 。
'001455162_1.pdf' と '001455162_2.pdf' に分割しました 。


In [3]:
from google.colab import userdata
import os
import google.generativeai as genai
from PIL import Image
import sys
import json
import math
from typing import List, Dict, Any, Optional, Union
import fitz  # PyMuPDF
import time
from google.api_core import exceptions as core_exceptions # タイムアウトエラーなどを捕捉するため
from google.ai.generativelanguage import Tool, GoogleSearchRetrieval # グラウンディングに必要
from typing import List, Union, Dict, Any, Tuple
import concurrent.futures


#api_key = userdata.get('GOOGLE_API_KEY_88')
api_key = userdata.get('GOOGLE_API_KEY_4')

#MODEL_NAME = "gemini-1.5-flash"
#MODEL_NAME = "gemini-2.0-flash"
#MODEL_NAME = "gemini-2.0-flash-thinking-exp"
MODEL_NAME = 'gemini-2.5-pro-exp-03-25'

# Input PDF file
pdf_files = [
"001455149_1",
"001455149_2",
"001455152_1",
"001455152_2",
"001455159_1",
"001455159_2",
"001455160_1",
"001455160_2",
"001455161_1",
"001455161_2",
"001455162_1",
"001455162_2",
]


In [4]:
genai.configure(api_key=api_key)

# 生成パラメータ
GENERATION_CONFIG = genai.types.GenerationConfig(
    # APIからのレスポンス形式をJSONに指定
    response_mime_type="application/json",
    # 生成されるテキストのランダム性を制御 (0.0-1.0)
    temperature=0.2,
    # 生成に使用するトークンの累積確率を制御 (0.0-1.0)
    top_p=0.8
)

# --- グラウンディング用ツールの定義 ---
# Google検索リトリーバル用のツールを定義します。
# SDKの簡易メソッドを使用するのが簡単です。
# 引数に空の辞書を渡すことで、デフォルトのGoogle検索グラウンディングが有効になります。
google_search_retrieval_tool = Tool(
    google_search_retrieval=GoogleSearchRetrieval() # 空のGoogleSearchRetrievalオブジェクトを指定
)

# 出力ファイル名
OUTPUT_FILENAME = "gemini_vision_combined_output.json"


# プロンプト (JSON形式とスキーマの指示を含む)
PROMPT = """これらの画像に含まれる問題とその選択肢を分析し、解答を導き出してください。回答はJSON形式で出力してください。

最上位の階層はリスト形式とし、リストの各要素は一つの問題を表すオブジェクトとします。
各問題オブジェクトは、キーとして問+連番（例: "問1", "問2"）を一つだけ持ちます。
そのキーに対応する値はオブジェクトとし、以下のキーを持ちます:
- "理由": 解答に至る思考プロセスと詳細な解説（他の選択肢がなぜ誤りかの説明も含む）を文章で示してください。理由の中で改行をする場合は\nの代わりに\\nを使用してください。箇条書きや表は使用しないでください。
- "回答": 解答番号（選択肢の番号）を含む配列。解答が1つの場合も配列形式にしてください（例: [1] や [2, 4]）。

出力形式の例：
[
  {
    "問1": {
      "理由": "選択肢1が正解である根拠。選択肢2, 3, 4が不正解である理由...",
      "回答": [1]
    }
  },
  {
    "問2": {
      "理由": "XXXXXから、選択肢2が適切であると判断。他の選択肢が不適切な理由...",
      "回答": [2]
    }
  },
  {
    "問3": {
      "理由": "ZZZZZのYYYに着目し、複数の条件を満たす選択肢1と3が解答となる。\\n他の選択肢が不適切な理由...",
      "回答": [1, 3]
    }
  }
]

上記JSON形式に従って、与えられた画像に対する解答のみを出力してください。他の余計なテキスト（例: ```json ``` マークダウンなど）は含めないでください。
"""



In [5]:
def load_image(image_path: str) -> Optional[Image.Image]:
    """画像ファイルを安全に読み込む"""
    try:
        img = Image.open(image_path)
        # 必要であればここで画像形式やサイズのチェックを追加
        return img
    except FileNotFoundError:
        print(f"エラー: 画像ファイルが見つかりません: {image_path}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"エラー: 画像読み込み中に予期せぬエラー ({image_path}): {e}", file=sys.stderr)
        return None

def clean_json_string(raw_string: str) -> str:
    """API応答文字列からJSONマークダウンを除去し、前後の空白を削除する"""
    cleaned = raw_string.strip()
    if cleaned.startswith("```json"):
        cleaned = cleaned[7:]
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3]
    return cleaned.strip()


def call_gemini_vision_api(
    image_paths: List[str],
    prompt: str,
    model_name: str,
    generation_config: genai.types.GenerationConfig,
    use_grounding: bool = False, # <- グラウンディングを使用するかのフラグを追加
    timeout: int = 1200 # <- タイムアウトも引数で渡せるように変更
) -> Union[genai.types.GenerateContentResponse, Dict[str, Any]]: # <- 戻り値の型を明確化
    """
    画像パスのリストを受け取り、画像をロードしてGemini Vision APIを呼び出す。
    オプションでGoogle検索によるグラウンディングを有効にできる。
    成功時はAPIからの応答オブジェクト、失敗時はエラー情報を含む辞書を返す。

    Args:
        image_paths: 画像ファイルへのパスのリスト。
        prompt: モデルへの指示プロンプト。
        model_name: 使用するGeminiモデル名 (例: 'gemini-1.5-pro-latest')。
        generation_config: 生成時の設定 (温度、最大トークン数など)。
        use_grounding: Google検索によるグラウンディングを有効にするか (デフォルト: False)。
        timeout: API呼び出しのタイムアウト時間(秒) (デフォルト: 1200)。

    Returns:
        成功した場合: genai.types.GenerateContentResponse オブジェクト。
                   通常、応答テキストは response.text でアクセスできる。
                   グラウンディングが有効な場合、response.grounding_metadata も含まれる可能性がある。
        失敗した場合: エラー情報を含む辞書 (例: {"error": "...", "failed_paths": [...]})。
    """
    loaded_images: List[Image.Image] = []
    failed_loads: List[str] = []

    print(f"  チャンク内の画像 {len(image_paths)} 個を読み込み中...")
    for path in image_paths:
        img = load_image(path)
        if img:
            loaded_images.append(img)
        else:
            failed_loads.append(path)

    # プロンプトも画像もない場合はエラー（あるいはプロンプトのみは許容するかもしれない）
    if not prompt and not loaded_images:
         return {"error": "プロンプトも読み込める画像もありませんでした。", "failed_paths": failed_loads}
    # 画像パスが指定されたが、すべて読み込み失敗した場合
    elif image_paths and not loaded_images:
        return {"error": "チャンク内で読み込める画像がありませんでした。", "failed_paths": failed_loads}

    if failed_loads:
        print(f"  警告: このチャンクで {len(failed_loads)} 個の画像の読み込みに失敗しました。", file=sys.stderr)
        # print(f"  失敗したパス: {failed_loads}", file=sys.stderr) # 必要ならコメント解除

    try:
        # グラウンディングが有効な場合はツールリストを準備
        tools_list = [google_search_retrieval_tool] if use_grounding else None
        if use_grounding:
            print("  Google検索によるグラウンディングを有効にします。")

        model = genai.GenerativeModel(model_name)

        # プロンプトとロードした画像をリストとして結合
        # プロンプトが空でないことを確認してから追加
        content = []
        if prompt:
            content.append(prompt)
        content.extend(loaded_images) # 画像を追加

        image_count_text = f"{len(loaded_images)} 個の画像" if loaded_images else "画像なし"
        grounding_text = " (グラウンディング有効)" if use_grounding else ""
        print(f"  Gemini API ({model_name}) にプロンプトと{image_count_text}を送信しています...{grounding_text}")

        # API呼び出し時に tools 引数を追加
        response = model.generate_content(
            content,
            generation_config=generation_config,
            tools=tools_list, # <- ツールリストを渡す (Noneも可)
            request_options=genai.types.RequestOptions(timeout=timeout) # タイムアウト設定
        )

        # 完全な応答オブジェクトを返す
        return response

    except Exception as e:
        print(f"  エラー: Gemini API呼び出し中にエラーが発生しました: {e}", file=sys.stderr)
        # より詳細なエラー情報を返すようにする
        return {"error": f"Gemini API呼び出しエラー: {type(e).__name__}", "details": str(e)}

    except Exception as e:
        print(f"エラー: Gemini APIリクエスト中にエラーが発生しました: {e}", file=sys.stderr)
        # エラーレスポンスの詳細情報を取得試行 (元のコードより)
        error_details = f"APIリクエスト失敗 - {type(e).__name__}: {e}"
        if hasattr(e, 'response'):
            try:
                 # APIからのエラーレスポンス本文など、より詳細な情報を取得
                 error_details += f"\n  API Response: {e.response}"
                 # 安全性フィルターによるブロックなどの情報を確認
                 if hasattr(e.response, 'prompt_feedback'):
                     error_details += f"\n  プロンプトフィードバック: {e.response.prompt_feedback}"
                 if hasattr(e.response, 'candidates') and e.response.candidates:
                     candidate = e.response.candidates[0]
                     if hasattr(candidate, 'finish_reason'):
                         error_details += f"\n  完了理由: {candidate.finish_reason}"
                     if hasattr(candidate, 'safety_ratings'):
                         error_details += f"\n  安全性評価: {candidate.safety_ratings}"
            except Exception as inner_e:
                 error_details += f"\n  エラーレスポンス解析中の追加エラー: {inner_e}"
        # print(error_details, file=sys.stderr) # 詳細すぎる場合はコメントアウト
        return {"error": f"APIリクエスト失敗 - {type(e).__name__}", "details": str(e)}

    finally:
        # 読み込んだ画像オブジェクトのメモリを解放
        if loaded_images:
             print(f"  チャンクの画像オブジェクト {len(loaded_images)} 個を解放します。")
             for img in loaded_images:
                 try:
                     img.close()
                 except Exception as e_close:
                     # 解放時のエラーは警告にとどめる
                     print(f"  警告: 画像解放中にエラーが発生しました: {e_close}", file=sys.stderr)
             # リスト自体の参照も削除（ガベージコレクションを助ける）
             del loaded_images

def merge_question_lists_indexed(list_of_lists):
  """
  リストのリストとして提供される複数の質問リストを結合します。
  重複する問がある場合、最初に現れたリストの問を保持し、
  重複した問の情報（どのリストに含まれていたか）をエラーリストとして返します。

  Args:
    list_of_lists (list): 結合したい質問リストを含むリスト。
                          例: [[リストA], [リストB], ...]
                          各内部リストは {"問X": {"理由": "...", "回答": [...]}}
                          形式の辞書のリスト。

  Returns:
    tuple: (merged_list, error_list)
      - merged_list (list): 重複を除いて結合された質問リスト。
      - error_list (list): 重複していた問の情報を含む辞書のリスト。
                             各辞書は {"list_index": int, "question_id": str} の形式。
                             list_index は重複が見つかった要素が含まれるリストの、
                             list_of_lists 内でのインデックス（0始まり）。
  """
  merged_data = {}  # 結合結果を一時的に格納する辞書 {問ID: 内容}
  errors = []       # 重複エラー情報を格納するリスト

  # 入力がリストのリストであることを確認（任意）
  if not isinstance(list_of_lists, list) or not all(isinstance(sublist, list) for sublist in list_of_lists):
      raise TypeError("入力はリストのリストである必要があります。例: [[...], [...]]")

  for list_index, current_list in enumerate(list_of_lists):
    for item in current_list:
      # 各辞書アイテムから問ID（キー）と内容（バリュー）を取得
      if not isinstance(item, dict) or not item: # アイテムが辞書でない、または空の辞書はスキップ
          print(f"警告: リスト {list_index} に不正な形式のアイテムが見つかりました。スキップします: {item}")
          continue
      try:
          # 辞書が {"問X": ...} の形式であることを想定
          if len(item) != 1:
              raise ValueError("各アイテムの辞書はキーを1つだけ持つ必要があります。")
          question_id = list(item.keys())[0]
          question_content = item[question_id]
          # 内容が辞書で、'理由'と'回答'キーを持つことを確認（より厳密にする場合）
          if not isinstance(question_content, dict) or '理由' not in question_content or '回答' not in question_content:
               raise ValueError(f"問 '{question_id}' の内容の形式が不正です。")

      except (IndexError, ValueError, KeyError) as e:
          print(f"警告: リスト {list_index} のアイテム処理中にエラーが発生しました。スキップします: {item} (エラー: {e})")
          continue

      # 既に同じ問IDが登録されているかチェック
      if question_id in merged_data:
        # 重複している場合、エラーリストに追加
        # この問がどのリストに含まれていたかの情報を記録
        errors.append({
            "list_index": list_index,
            "question_id": question_id
        })
        # merged_data は更新しない（既存のものが保持される）
      else:
        # 新しい問IDの場合、結合結果辞書に追加
        merged_data[question_id] = question_content

  # 結合結果辞書を元のリスト形式（辞書のリスト）に戻す
  # 順序は保証されないが、通常は問題ない。もし順序維持が必要ならOrderedDictを使う
  final_merged_list = [{qid: data} for qid, data in merged_data.items()]

  return final_merged_list, errors


In [6]:

# 設定値
MAX_CONCURRENT_WORKERS = 4
MAX_RETRIES = 3  # 最大リトライ回数 (0ならリトライしない)
RETRY_DELAY_SECONDS = 10 # リトライ間の待機時間（秒）

# --- process_single_pdf 関数 ---
# (前回のコード例と同じ、エラー時に {"error": ..., "pdf_file": ...} を返すことを確認)
def process_single_pdf(pdf_file_base: str, index: int, total_files: int) -> Dict[str, Any]:
    """
    Processes a single PDF file: converts pages to images and calls the Gemini API.
    Returns a dictionary containing the result or error information.
    """
    pdf_path = pdf_file_base + '.pdf'
    output_dir = pdf_file_base
    run_identifier = f"[ファイル {index + 1}/{total_files}] PDF: {os.path.basename(pdf_path)}"

    print(f"\n{run_identifier} の処理開始")

    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"{run_identifier} - PDFオープンエラー: {e}")
        # index と pdf_file_base を含めて返す
        return {"index": index, "pdf_file": pdf_file_base, "error": f"Failed to open PDF: {e}"}

    Num = 0
    PicsPath = []
    os.makedirs(output_dir, exist_ok=True)
    print(f"  {run_identifier} - 画像変換中 -> '{output_dir}'")

    try:
        for page_num in range(doc.page_count):
            page = doc[page_num]
            pix = page.get_pixmap()
            output_filename = os.path.join(output_dir, f"{Num:03d}.png")
            pix.save(output_filename)
            Num += 1
            PicsPath.append(output_filename)
        doc.close()
    except Exception as e:
        print(f"  {run_identifier} - 画像変換エラー: {e}")
        try:
            doc.close()
        except:
            pass
        return {"index": index, "pdf_file": pdf_file_base, "error": f"Failed during image conversion: {e}"}

    print(f"  {run_identifier} - 画像 {len(PicsPath)} 個のAPI処理開始")
    print("  " + "-" * 28)
    start_time = time.time()

    try:
        response = call_gemini_vision_api(
            PicsPath,
            PROMPT,
            MODEL_NAME,
            GENERATION_CONFIG,
        )
        parsed_response = json.loads(response.text)
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"  {run_identifier} - 結果生成完了 ({elapsed_time:.4f} 秒)")
        if hasattr(response, 'usage_metadata') and response.usage_metadata:
            print(f"    Token Count: P={response.usage_metadata.prompt_token_count}, T={response.usage_metadata.total_token_count}")
        else:
            print("    Token usage metadata not available.")
        print("  " + "-" * 28)
        # index と response を含めて返す
        return {"index": index, "pdf_file": pdf_file_base, "response": parsed_response}

    except Exception as e:
        print(f"  {run_identifier} - API呼び出しエラー: {e}")
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"    試行時間: {elapsed_time:.4f} 秒")
        print("  " + "-" * 28)
        # index, pdf_file_base, error を含めて返す
        return {"index": index, "pdf_file": pdf_file_base, "error": f"API call failed: {e}"}

# --- メインの実行部分 ---
print("--- Gemini Vision APIによる画像処理を開始します ---")

# pdf_files = ["doc1", "doc2_fail_once", "doc3", "doc4_invalid"] # Example list

all_results_with_indices = {} # 辞書を使ってindexをキーに結果を格納（重複実行を防ぐため）
files_to_process = list(enumerate(pdf_files)) # (index, pdf_file_base) のリスト
retry_attempt = 0
failed_files_info = [] # リトライが必要なファイルの情報を格納: List[Tuple[int, str]] = (index, pdf_file_base)

while retry_attempt <= MAX_RETRIES:
    if retry_attempt > 0:
        print(f"\n--- リトライ試行 {retry_attempt}/{MAX_RETRIES} 開始 ({len(files_to_process)} 件) ---")
        if RETRY_DELAY_SECONDS > 0:
             print(f"  {RETRY_DELAY_SECONDS}秒待機...")
             time.sleep(RETRY_DELAY_SECONDS)

    current_failed_files = [] # この試行で失敗したファイル

    # ProcessPoolExecutorを使う場合はThreadPoolExecutorを置き換える
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_CONCURRENT_WORKERS) as executor:
        # 各PDFファイルに対する処理タスクを作成
        future_to_info = {executor.submit(process_single_pdf, pdf_base, idx, len(pdf_files)): (idx, pdf_base) for idx, pdf_base in files_to_process}

        # タスクの完了を待機し、結果を収集
        for future in concurrent.futures.as_completed(future_to_info):
            original_index, pdf_base = future_to_info[future]
            try:
                # タスクの結果（process_single_pdfの戻り値）を取得
                result = future.result()
                # 成功か失敗かを判断し、結果を格納
                if "error" not in result:
                    # 成功した場合、元のindexをキーとして結果を保存
                    if original_index not in all_results_with_indices:
                         all_results_with_indices[original_index] = result
                else:
                    # 失敗した場合
                    print(f"  ファイル {pdf_base}.pdf (Index: {original_index}) の処理に失敗。エラー: {result['error']}")
                    current_failed_files.append((original_index, pdf_base)) # 次のリトライ候補に追加

            except Exception as exc:
                # process_single_pdf内でキャッチされなかった予期せぬエラー
                print(f'ファイル {pdf_base}.pdf (Index: {original_index}) で予期せぬ実行時エラー発生: {exc}')
                # 予期せぬエラーもリトライ候補に追加
                current_failed_files.append((original_index, pdf_base))
                # エラー情報を記録しておく（任意）
                if original_index not in all_results_with_indices: # まだ結果がなければエラーとして記録
                    all_results_with_indices[original_index] = {"index": original_index, "pdf_file": pdf_base, "error": f"Unexpected execution error: {exc}"}


    # 次のリトライのために失敗リストを更新
    failed_files_info = current_failed_files
    files_to_process = failed_files_info # 次のループでは失敗したファイルのみ処理

    if not failed_files_info:
        print(f"\n--- 試行 {retry_attempt} で全ての処理が成功または完了 ---")
        break # 失敗がなければループを抜ける

    retry_attempt += 1

# --- ループ終了後の処理 ---

# 最終的に失敗したファイルがあれば報告
final_failed_files = failed_files_info # ループを抜けた時点での失敗リスト
if final_failed_files:
    print(f"\n--- {MAX_RETRIES}回のリトライ後もエラーが解消しなかったファイル ---")
    for index, pdf_base in final_failed_files:
         # all_results_with_indices から最後のエラーメッセージを取得試行
        last_error_info = all_results_with_indices.get(index, {})
        error_msg = last_error_info.get("error", "Unknown error during final attempt")
        print(f"  - ファイル: {pdf_base}.pdf (Index: {index}), 最終エラー: {error_msg}")

# 結果を元の順序にソートするためにインデックスでソート
# all_results_with_indices には成功した結果と、最終的に失敗したファイルのエラー情報が含まれる
sorted_indices = sorted(all_results_with_indices.keys())
sorted_results = [all_results_with_indices[i] for i in sorted_indices]

# 実際のレスポンス（エラーでないもの）だけを抽出してマージ対象とする
successful_api_responses = [r.get("response") for r in sorted_results if "response" in r and "error" not in r]

# 全ての成功したチャンクの応答を統合・整形
merged_list, merge_error_info = merge_question_lists_indexed(successful_api_responses)

print("\n--- 結合結果 (成功分) ---")
print("結合されたリスト:")
marged = json.dumps(merged_list, indent=2, ensure_ascii=False, sort_keys=True)
print(marged)

print("\n--- 重複エラー情報 (成功分) ---")
if merge_error_info:
  print("以下の問が重複していました（最初に現れたものが保持されます）：")
  for error in merge_error_info:
    print(f"- リスト {error['list_index']} に含まれる '{error['question_id']}'") # インデックスは成功リスト内でのもの
else:
  print("重複エラーはありませんでした。")

# 成功した結果のみファイルに保存
if successful_api_responses:
    with open(OUTPUT_FILENAME, "w", encoding="utf-8") as f:
        f.write(marged)
    print(f"\n成功した結果を '{OUTPUT_FILENAME}' に保存しました。")
else:
    print(f"\n成功した結果がなかったため、'{OUTPUT_FILENAME}' は作成されませんでした。")


print("\n--- 全ての処理が完了しました ---")

--- Gemini Vision APIによる画像処理を開始します ---

[ファイル 1/12] PDF: 001455149_1.pdf の処理開始
  [ファイル 1/12] PDF: 001455149_1.pdf - 画像変換中 -> '001455149_1'

[ファイル 2/12] PDF: 001455149_2.pdf の処理開始

[ファイル 3/12] PDF: 001455152_1.pdf の処理開始
  [ファイル 3/12] PDF: 001455152_1.pdf - 画像変換中 -> '001455152_1'
  [ファイル 2/12] PDF: 001455149_2.pdf - 画像変換中 -> '001455149_2'

[ファイル 4/12] PDF: 001455152_2.pdf の処理開始
  [ファイル 4/12] PDF: 001455152_2.pdf - 画像変換中 -> '001455152_2'
  [ファイル 2/12] PDF: 001455149_2.pdf - 画像 18 個のAPI処理開始  [ファイル 1/12] PDF: 001455149_1.pdf - 画像 20 個のAPI処理開始
  ----------------------------
  チャンク内の画像 20 個を読み込み中...

  ----------------------------
  チャンク内の画像 18 個を読み込み中...
  [ファイル 4/12] PDF: 001455152_2.pdf - 画像 24 個のAPI処理開始
  ----------------------------
  チャンク内の画像 24 個を読み込み中...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと18 個の画像を送信しています...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと24 個の画像を送信しています...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと20 個の画像を送信しています...
  [ファイル 3/12] PDF: 001455152_1.pdf -

  エラー: Gemini API呼び出し中にエラーが発生しました: HTTPConnectionPool(host='localhost', port=42197): Read timed out. (read timeout=1200.0)


  チャンクの画像オブジェクト 26 個を解放します。
  [ファイル 3/12] PDF: 001455152_1.pdf - API呼び出しエラー: 'dict' object has no attribute 'text'
    試行時間: 1200.4047 秒
  ----------------------------

[ファイル 8/12] PDF: 001455160_2.pdf の処理開始
  ファイル 001455152_1.pdf (Index: 2) の処理に失敗。エラー: API call failed: 'dict' object has no attribute 'text'
  [ファイル 8/12] PDF: 001455160_2.pdf - 画像変換中 -> '001455160_2'
  [ファイル 8/12] PDF: 001455160_2.pdf - 画像 26 個のAPI処理開始
  ----------------------------
  チャンク内の画像 26 個を読み込み中...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと26 個の画像を送信しています...


  エラー: Gemini API呼び出し中にエラーが発生しました: HTTPConnectionPool(host='localhost', port=42197): Read timed out. (read timeout=1200.0)


  チャンクの画像オブジェクト 17 個を解放します。
  [ファイル 5/12] PDF: 001455159_1.pdf - API呼び出しエラー: 'dict' object has no attribute 'text'
    試行時間: 1200.1206 秒
  ----------------------------

[ファイル 9/12] PDF: 001455161_1.pdf の処理開始
  ファイル 001455159_1.pdf (Index: 4) の処理に失敗。エラー: API call failed: 'dict' object has no attribute 'text'
  [ファイル 9/12] PDF: 001455161_1.pdf - 画像変換中 -> '001455161_1'
  [ファイル 9/12] PDF: 001455161_1.pdf - 画像 21 個のAPI処理開始
  ----------------------------
  チャンク内の画像 21 個を読み込み中...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと21 個の画像を送信しています...


...

  チャンクの画像オブジェクト 26 個を解放します。
  [ファイル 8/12] PDF: 001455160_2.pdf - API呼び出しエラー: 'dict' object has no attribute 'text'
    試行時間: 1200.1175 秒
  ----------------------------

[ファイル 12/12] PDF: 001455162_2.pdf の処理開始
  ファイル 001455160_2.pdf (Index: 7) の処理に失敗。エラー: API call failed: 'dict' object has no attribute 'text'
  [ファイル 12/12] PDF: 001455162_2.pdf - 画像変換中 -> '001455162_2'
  [ファイル 12/12] PDF: 001455162_2.pdf - 画像 31 個のAPI処理開始
  ----------------------------
  チャンク内の画像 31 個を読み込み中...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと31 個の画像を送信しています...
  チャンクの画像オブジェクト 21 個を解放します。
  [ファイル 9/12] PDF: 001455161_1.pdf - 結果生成完了 (1020.6086 秒)
    Token Count: P=5836, T=19532
  ----------------------------
  チャンクの画像オブジェクト 31 個を解放します。
  [ファイル 11/12] PDF: 001455162_1.pdf - 結果生成完了 (949.3756 秒)
    Token Count: P=8416, T=36354
  ----------------------------


  エラー: Gemini API呼び出し中にエラーが発生しました: HTTPConnectionPool(host='localhost', port=42197): Read timed out. (read timeout=1200.0)


  チャンクの画像オブジェクト 22 個を解放します。
  [ファイル 10/12] PDF: 001455161_2.pdf - API呼び出しエラー: 'dict' object has no attribute 'text'
    試行時間: 1200.1231 秒
  ----------------------------
  ファイル 001455161_2.pdf (Index: 9) の処理に失敗。エラー: API call failed: 'dict' object has no attribute 'text'
  チャンクの画像オブジェクト 31 個を解放します。
  [ファイル 12/12] PDF: 001455162_2.pdf - 結果生成完了 (1129.2133 秒)
    Token Count: P=8416, T=42744
  ----------------------------

--- リトライ試行 1/3 開始 (5 件) ---
  10秒待機...

[ファイル 3/12] PDF: 001455152_1.pdf の処理開始
  [ファイル 3/12] PDF: 001455152_1.pdf - 画像変換中 -> '001455152_1'

[ファイル 5/12] PDF: 001455159_1.pdf の処理開始

[ファイル 6/12] PDF: 001455159_2.pdf の処理開始
  [ファイル 6/12] PDF: 001455159_2.pdf - 画像変換中 -> '001455159_2'
  [ファイル 5/12] PDF: 001455159_1.pdf - 画像変換中 -> '001455159_1'

[ファイル 8/12] PDF: 001455160_2.pdf の処理開始
  [ファイル 8/12] PDF: 001455160_2.pdf - 画像変換中 -> '001455160_2'
  [ファイル 6/12] PDF: 001455159_2.pdf - 画像 14 個のAPI処理開始
  ----------------------------
  チャンク内の画像 14 個を読み込み中...
  [ファイル 5/12] PDF: 001455159_1

  エラー: Gemini API呼び出し中にエラーが発生しました: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))



[ファイル 10/12] PDF: 001455161_2.pdf の処理開始
  [ファイル 10/12] PDF: 001455161_2.pdf - 画像変換中 -> '001455161_2'

[ファイル 8/12] PDF: 001455160_2.pdf の処理開始
  [ファイル 8/12] PDF: 001455160_2.pdf - 画像変換中 -> '001455160_2'
  [ファイル 10/12] PDF: 001455161_2.pdf - 画像 22 個のAPI処理開始
  ----------------------------
  チャンク内の画像 22 個を読み込み中...
  [ファイル 8/12] PDF: 001455160_2.pdf - 画像 26 個のAPI処理開始
  ----------------------------
  チャンク内の画像 26 個を読み込み中...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと22 個の画像を送信しています...
  Gemini API (gemini-2.5-pro-exp-03-25) にプロンプトと26 個の画像を送信しています...
  チャンクの画像オブジェクト 26 個を解放します。
  [ファイル 8/12] PDF: 001455160_2.pdf - 結果生成完了 (287.7377 秒)
    Token Count: P=7126, T=35611
  ----------------------------
  チャンクの画像オブジェクト 22 個を解放します。
  [ファイル 10/12] PDF: 001455161_2.pdf - 結果生成完了 (481.7975 秒)
    Token Count: P=6094, T=23995
  ----------------------------

--- 試行 2 で全ての処理が成功または完了 ---

--- 結合結果 (成功分) ---
結合されたリスト:
[
  {
    "問1": {
      "回答": [
        3
      ],
      "理由": "ダニエル電池の起電力は、正極（銅）の標準電極電位か

In [7]:
#print(response.text)

In [8]:
AnswerData = {
    "問1"  : [(3,  )],  "問2"  : [(4,  )],  "問3"  : [(5,  )],  "問4"  : [(5,  )],  "問5"  : [(1,  )],  "問6"  : [(4,  )],  "問7"  : [(3,  )],  "問8"  : [(5,  )],  "問9"  : [(4,  )],  "問10" : [(3,  )],
    "問11" : [(3,  )],  "問12" : [(5,  )],  "問13" : [(3,  )],  "問14" : [(5,  )],  "問15" : [(2,  )],  "問16" : [(4,  )],  "問17" : [(1,  )],  "問18" : [(3,  )],  "問19" : [(3,  )],  "問20" : [(3,  )],
    "問21" : [(1,  )],  "問22" : [(4,  )],  "問23" : [(4,  )],  "問24" : [(2,  )],  "問25" : [(2,  )],  "問26" : [(4,  )],  "問27" : [(1,  )],  "問28" : [(5,  )],  "問29" : [(1,  )],  "問30" : [(3,  )],
    "問31" : [(4,  )],  "問32" : [(4,  )],  "問33" : [(5,  )],  "問34" : [(1,  )],  "問35" : [(5,  )],  "問36" : [(3,  )],  "問37" : [(2,  )],  "問38" : [(3,  )],  "問39" : [(3,  )],  "問40" : [(2,  )],
    "問41" : [(5,  )],  "問42" : [(4,  )],  "問43" : [(5,  )],  "問44" : [(2,  )],  "問45" : [(3,  )],  "問46" : [(4,  )],  "問47" : [(4,  )],  "問48" : [(4,  )],  "問49" : [(5,  )],  "問50" : [(1,  )],
    "問51" : [(2,  )],  "問52" : [(2,  )],  "問53" : [(1,  )],  "問54" : [(3,  )],  "問55" : [(1,  )],  "問56" : [(2,  )],  "問57" : [(2,  )],  "問58" : [(4,  )],  "問59" : [(1,  )],  "問60" : [(5,  )],
    "問61" : [(1,  )],  "問62" : [(3,  )],  "問63" : [(1,  )],  "問64" : [(2,  )],  "問65" : [(1,  )],  "問66" : [(1,  )],  "問67" : [(4,  )],  "問68" : [(3,  )],  "問69" : [(4,  )],  "問70" : [(1,  )],
    "問71" : [(2,  )],  "問72" : [(3,  )],  "問73" : [(2,  )],  "問74" : [(3,  )],  "問75" : [(5,  )],  "問76" : [(2,  )],  "問77" : [(1,  )],  "問78" : [(3,  )],  "問79" : [(2,  )],  "問80" : [(2,  )],
    "問81" : [(5,  )],  "問82" : [(4,  )],  "問83" : [(2,  )],  "問84" : [(1,  )],  "問85" : [(5,  )],  "問86" : [(5,  )],  "問87" : [(4,  )],  "問88" : [(4,  )],  "問89" : [(3,  )],  "問90" : [(2,  )],
    "問91" : [(1, 2)],  "問92" : [(1, 4)],  "問93" : [(3,  )],  "問94" : [(1, 2)],  "問95" : [(1, 3)],  "問96" : [(4,  )],  "問97" : [(2,  )],  "問98" : [(2, 5)],  "問99" : [(1, 4)],  "問100": [(2, 4)],
    "問101": [(6,  )],  "問102": [(4,  )],  "問103": [(3, 4)],  "問104": [(4, 5)],  "問105": [(1, 2)],  "問106": [(1,  )],  "問107": [(2,  )],  "問108": [(2, 4)],  "問109": [(2, 3)],  "問110": [(4,  )],
    "問111": [(1, 3)],  "問112": [(3,  )],  "問113": [(3, 4)],  "問114": [(3, 5)],  "問115": [(2, 3)],  "問116": [(2,  )],  "問117": [(3,  )],  "問118": [(1, 4)],  "問119": [(1, 3), (1, 5), (3, 5)],  "問120": [(2, 4)],
    "問121": [(2, 3)],  "問122": [(3, 5)],  "問123": [(2, 6)],  "問124": [(4, 5)],  "問125": [(3, 5)],  "問126": [(3, 5)],  "問127": [(1, 4)],  "問128": [(1, 3)],  "問129": [(3, 4)],  "問130": [(1, 3)],
    "問131": [(1, 3)],  "問132": [(1, 4)],  "問133": [(3, 5)],  "問134": [(1, 5)],  "問135": [(1, 5)],  "問136": [(1, 4)],  "問137": [(1, 4)],  "問138": [(1, 4)],  "問139": [(2, 3)],  "問140": [(2, 5)],
    "問141": [(1, 2)],  "問142": [(3, 4)],  "問143": [(3, 4)],  "問144": [(1, 2)],  "問145": [(1, 4)],  "問146": [(4, 5)],  "問147": [(1, 2)],  "問148": [(5  ,)],  "問149": [(3, 5)],  "問150": [(3, 4)],
    "問151": [(1, 2)],  "問152": [(2, 5)],  "問153": [(3, 4)],  "問154": [(1, 4)],  "問155": [(2, 5)],  "問156": [(1, 4)],  "問157": [(2, 4)],  "問158": [(2, 4)],  "問159": [(3, 4)],  "問160": [(1, 5)],
    "問161": [(1, 2)],  "問162": [(1, 5)],  "問163": [(3, 4)],  "問164": [(1, 5)],  "問165": [(2, 4)],  "問166": [(3, 4)],  "問167": [(1, 3)],  "問168": [(1,  )],  "問169": [(5,  )],  "問170": [(3,  )],
    "問171": [(1,  )],  "問172": [(2, 4)],  "問173": [(3, 5)],  "問174": [(3, 4)],  "問175": [(1, 3)],  "問176": [(3,  )],  "問177": [(2, 5)],  "問178": [(1, 3)],  "問179": [(3, 5)],  "問180": [(1, 4)],
    "問181": [(3,  )],  "問182": [(2, 3)],  "問183": [(4, 5)],  "問184": [(2, 5)],  "問185": [(1, 3)],  "問186": [(2, 4)],  "問187": [(2, 4)],  "問188": [(2, 5)],  "問189": [(1, 3)],  "問190": [(1, 3)],
    "問191": [(1, 3)],  "問192": [(2, 5)],  "問193": [(2, 4)],  "問194": [(3,  )],  "問195": [(1, 3)],  "問196": [(4,  )],  "問197": [(3,  )],  "問198": [(1, 2)],  "問199": [(3,  )],  "問200": [(2, 4)],
    "問201": [(3,  )],  "問202": [(3,  )],  "問203": [(1, 4)],  "問204": [(2, 4)],  "問205": [(2, 3)],  "問206": [(3,  )],  "問207": [(3, 4)],  "問208": [(2, 3)],  "問209": [(1,  )],  "問210": [(4,  )],
    "問211": [(3,  )],  "問212": [(5,  )],  "問213": [(2, 4)],  "問214": [(2, 4)],  "問215": [(4,  )],  "問216": [(1, 2)],  "問217": [(4,  )],  "問218": [(3, 4)],  "問219": [(1, 4)],  "問220": [(2, 5)],
    "問221": [(2, 5)],  "問222": [(3, 5)],  "問223": [(1, 2)],  "問224": [(4,  )],  "問225": [(3,  )],  "問226": [(2, 3)],  "問227": [(5,  )],  "問228": [(3, 5)],  "問229": [(1,  )],  "問230": [(1, 3)],
    "問231": [(2, 3)],  "問232": [(1,  )],  "問233": [(4, 5)],  "問234": [(1, 3)],  "問235": [(5,  )],  "問236": [(1, 4)],  "問237": [(4,  )],  "問238": [(4,  )],  "問239": [(2, 5)],  "問240": [(1, 5)],
    "問241": [(2, 5)],  "問242": [(3,  )],  "問243": [(3, 5)],  "問244": [(2, 5)],  "問245": [(1, 3)],  "問246": [(3, 5)],  "問247": [(4,  )],  "問248": [(1, 4)],  "問249": [(4, 5)],  "問250": [(1, 5)],
    "問251": [(3, 5)],  "問252": [(2, 3)],  "問253": [(1, 3)],  "問254": [(3,  )],  "問255": [(3, 4)],  "問256": [(4, 5)],  "問257": [(4,  )],  "問258": [(3,  )],  "問259": [(2,  )],  "問260": [(2, 3)],
    "問261": [(2, 5)],  "問262": [(2, 3)],  "問263": [(2, 5)],  "問264": [(2, 5)],  "問265": [(1, 2)],  "問266": [(2,  )],  "問267": [(1, 4)],  "問268": [(1, 3)],  "問269": [(2,  )],  "問270": [(5,  )],
    "問271": [(3,  )],  "問272": [(3, 5)],  "問273": [(1, 3)],  "問274": [(4,  )],  "問275": [(2, 5)],  "問276": [(2, 3)],  "問277": [(1, 2)],  "問278": [(3, 4)],  "問279": [(4,  )],  "問280": [(1, 5)],
    "問281": [(1,  )],  "問282": [(1, 4)],  "問283": [(1, 3)],  "問284": [(2, 5)],  "問285": [(3, 5)],  "問286": [(1, 3)],  "問287": [(3, 5)],  "問288": [(1, 4)],  "問289": [(1,  )],  "問290": [(2,  )],
    "問291": [(2,  )],  "問292": [(3, 5)],  "問293": [(2, 3)],  "問294": [(1, 2)],  "問295": [(1, 5)],  "問296": [(4, 5)],  "問297": [(2, 4)],  "問298": [(1, 2)],  "問299": [(2, 4)],  "問300": [(4, 5)],
    "問301": [(4,  )],  "問302": [(1, 3)],  "問303": [(3, 4)],  "問304": [(2, 3)],  "問305": [(1, 4)],  "問306": [(1, 2)],  "問307": [(1, 5)],  "問308": [(1, 3)],  "問309": [(1, 5)],  "問310": [(2, 5)],
    "問311": [(4,  )],  "問312": [(1, 5)],  "問313": [(3, 5)],  "問314": [(2, 4)],  "問315": [(5,  )],  "問316": [(1, 3)],  "問317": [(3, 5)],  "問318": [(2, 4)],  "問319": [(4, 5)],  "問320": [(3, 4)],
    "問321": [(2,  )],  "問322": [(3, 5)],  "問323": [(4, 5)],  "問324": [(1, 2)],  "問325": [(2,  )],  "問326": [(3,  )],  "問327": [(2,  )],  "問328": [(2, 4)],  "問329": [(3,  )],  "問330": [(1, 4)],
    "問331": [(1, 2), (1, 5), (2, 5)],  "問332": [(1, 4)],  "問333": [(5,  )],  "問334": [(4, 5)],  "問335": [(3,  )],  "問336": [(4, 5)],  "問337": [(1, 3)],  "問338": [(3,  )],  "問339": [(4,  )],  "問340": [(2, 4)],
    "問341": [(4, 5)],  "問342": [(2, 4)],  "問343": [(2, 3)],  "問344": [(3,  )],  "問345": [(2, 4)]
}

In [9]:
Category = {
    "問1"  : "物理",  "問2"  : "物理",  "問3"  : "物理",  "問4"  : "物理",  "問5"  : "物理",  "問6"  : "化学",  "問7"  : "化学",  "問8"  : "化学",  "問9"  : "化学",  "問10" : "化学",
    "問11" : "生物",  "問12" : "生物",  "問13" : "生物",  "問14" : "生物",  "問15" : "生物",  "問16" : "衛生",  "問17" : "衛生",  "問18" : "衛生",  "問19" : "衛生",  "問20" : "衛生",
    "問21" : "衛生",  "問22" : "衛生",  "問23" : "衛生",  "問24" : "衛生",  "問25" : "衛生",  "問26" : "薬理",  "問27" : "薬理",  "問28" : "薬理",  "問29" : "薬理",  "問30" : "薬理",
    "問31" : "薬理",  "問32" : "薬理",  "問33" : "薬理",  "問34" : "薬理",  "問35" : "薬理",  "問36" : "薬理",  "問37" : "薬理",  "問38" : "薬理",  "問39" : "薬理",  "問40" : "薬理",
    "問41" : "薬剤",  "問42" : "薬剤",  "問43" : "薬剤",  "問44" : "薬剤",  "問45" : "薬剤",  "問46" : "薬剤",  "問47" : "薬剤",  "問48" : "薬剤",  "問49" : "薬剤",  "問50" : "薬剤",
    "問51" : "薬剤",  "問52" : "薬剤",  "問53" : "薬剤",  "問54" : "薬剤",  "問55" : "薬剤",  "問56" : "病態",  "問57" : "病態",  "問58" : "病態",  "問59" : "病態",  "問60" : "病態",
    "問61" : "病態",  "問62" : "病態",  "問63" : "病態",  "問64" : "病態",  "問65" : "病態",  "問66" : "病態",  "問67" : "病態",  "問68" : "病態",  "問69" : "病態",  "問70" : "病態",
    "問71" : "法規",  "問72" : "法規",  "問73" : "法規",  "問74" : "法規",  "問75" : "法規",  "問76" : "法規",  "問77" : "法規",  "問78" : "法規",  "問79" : "法規",  "問80" : "法規",
    "問81" : "実務",  "問82" : "実務",  "問83" : "実務",  "問84" : "実務",  "問85" : "実務",  "問86" : "実務",  "問87" : "実務",  "問88" : "実務",  "問89" : "実務",  "問90" : "実務",
    "問91" : "物理",  "問92" : "物理",  "問93" : "物理",  "問94" : "物理",  "問95" : "物理",  "問96" : "物理",  "問97" : "物理",  "問98" : "物理",  "問99" : "物理",  "問100": "物理",
    "問101": "化学",  "問102": "化学",  "問103": "化学",  "問104": "化学",  "問105": "化学",  "問106": "化学",  "問107": "化学",  "問108": "化学",  "問109": "化学",  "問110": "化学",
    "問111": "生物",  "問112": "生物",  "問113": "生物",  "問114": "生物",  "問115": "生物",  "問116": "生物",  "問117": "生物",  "問118": "生物",  "問119": "生物",  "問120": "衛生",
    "問121": "生物",  "問122": "衛生",  "問123": "衛生",  "問124": "衛生",  "問125": "衛生",  "問126": "衛生",  "問127": "衛生",  "問128": "衛生",  "問129": "衛生",  "問130": "衛生",
    "問131": "衛生",  "問132": "衛生",  "問133": "衛生",  "問134": "衛生",  "問135": "衛生",  "問136": "衛生",  "問137": "衛生",  "問138": "衛生",  "問139": "衛生",  "問140": "衛生",
    "問141": "法規",  "問142": "法規",  "問143": "法規",  "問144": "法規",  "問145": "法規",  "問146": "法規",  "問147": "法規",  "問148": "法規",  "問149": "法規",  "問150": "法規",
    "問151": "薬理",  "問152": "薬理",  "問153": "薬理",  "問154": "薬理",  "問155": "薬理",  "問156": "薬理",  "問157": "病態",  "問158": "薬理",  "問159": "薬理",  "問160": "薬理",
    "問161": "薬理",  "問162": "病態",  "問163": "薬理",  "問164": "薬理",  "問165": "病態",  "問166": "薬理",  "問167": "薬理",  "問168": "薬理",  "問169": "薬剤",  "問170": "薬剤",
    "問171": "薬剤",  "問172": "薬剤",  "問173": "薬剤",  "問174": "薬剤",  "問175": "薬剤",  "問176": "薬剤",  "問177": "薬剤",  "問178": "薬剤",  "問179": "薬剤",  "問180": "薬剤",
    "問181": "薬剤",  "問182": "薬剤",  "問183": "薬剤",  "問184": "病態",  "問185": "病態",  "問186": "病態",  "問187": "病態",  "問188": "病態",  "問189": "病態",  "問190": "病態",
    "問191": "病態",  "問192": "病態",  "問193": "病態",  "問194": "病態",  "問195": "病態",  "問196": "物理",  "問197": "実務",  "問198": "実務",  "問199": "物理",  "問200": "実務",
    "問201": "物理",  "問202": "物理",  "問203": "実務",  "問204": "物理",  "問205": "実務",  "問206": "衛生",  "問207": "実務",  "問208": "実務",  "問209": "化学",  "問210": "化学",
    "問211": "実務",  "問212": "化学",  "問213": "実務",  "問214": "実務",  "問215": "化学",  "問216": "実務",  "問217": "化学",  "問218": "実務",  "問219": "生物",  "問220": "実務",
    "問221": "生物",  "問222": "生物",  "問223": "実務",  "問224": "実務",  "問225": "生物",  "問226": "実務",  "問227": "生物",  "問228": "実務",  "問229": "衛生",  "問230": "衛生",
    "問231": "実務",  "問232": "実務",  "問233": "衛生",  "問234": "衛生",  "問235": "実務",  "問236": "実務",  "問237": "衛生",  "問238": "衛生",  "問239": "実務",  "問240": "実務",
    "問241": "衛生",  "問242": "衛生",  "問243": "実務",  "問244": "実務",  "問245": "衛生",  "問246": "薬理",  "問247": "実務",  "問248": "実務",  "問249": "薬理",  "問250": "薬理",
    "問251": "実務",  "問252": "実務",  "問253": "薬理",  "問254": "薬理",  "問255": "実務",  "問256": "薬理",  "問257": "実務",  "問258": "実務",  "問259": "薬理",  "問260": "実務",
    "問261": "薬理",  "問262": "実務",  "問263": "薬理",  "問264": "実務",  "問265": "薬理",  "問266": "薬剤",  "問267": "実務",  "問268": "実務",  "問269": "薬剤",  "問270": "薬剤",
    "問271": "実務",  "問272": "薬剤",  "問273": "実務",  "問274": "薬剤",  "問275": "実務",  "問276": "薬剤",  "問277": "実務",  "問278": "実務",  "問279": "薬剤",  "問280": "実務",
    "問281": "薬剤",  "問282": "薬剤",  "問283": "実務",  "問284": "薬剤",  "問285": "実務",  "問286": "病態",  "問287": "実務",  "問288": "病態",  "問289": "実務",  "問290": "実務",
    "問291": "病態",  "問292": "病態",  "問293": "実務",  "問294": "病態",  "問295": "実務",  "問296": "病態",  "問297": "実務",  "問298": "実務",  "問299": "病態",  "問300": "病態",
    "問301": "実務",  "問302": "病態",  "問303": "実務",  "問304": "病態",  "問305": "実務",  "問306": "実務",  "問307": "法規",  "問308": "実務",  "問309": "法規",  "問310": "法規",
    "問311": "実務",  "問312": "実務",  "問313": "法規",  "問314": "実務",  "問315": "法規",  "問316": "実務",  "問317": "法規",  "問318": "実務",  "問319": "法規",  "問320": "法規",
    "問321": "実務",  "問322": "実務",  "問323": "法規",  "問324": "実務",  "問325": "法規",  "問326": "実務",  "問327": "実務",  "問328": "実務",  "問329": "実務",  "問330": "実務",
    "問331": "実務",  "問332": "実務",  "問333": "実務",  "問334": "実務",  "問335": "実務",  "問336": "実務",  "問337": "実務",  "問338": "実務",  "問339": "実務",  "問340": "実務",
    "問341": "実務",  "問342": "実務",  "問343": "実務",  "問344": "実務",  "問345": "実務"
}

In [10]:
import pandas as pd

with open('gemini_vision_combined_output.json', 'r') as f:
  Result = json.load(f)

# Resultデータを処理しやすいように辞書形式に変換
result_dict = {}
for item in Result:
    for key, value in item.items():
        result_dict[key] = value

# データフレーム用の中間リストを作成
df_data = []

# AnswerData_sampleのキー（問題番号）を基準にループ
for question, correct_answers in AnswerData.items():
    # 各情報を取得
    subject = Category.get(question, "N/A") # カテゴリが見つからない場合のデフォルト値
    user_data = result_dict.get(question, {"回答": [], "理由": ""}) # 結果が見つからない場合
    user_answer_raw = user_data.get("回答", [])
    reason = user_data.get("理由", "")

    # ユーザー回答をソートしてタプルに変換（比較用）
    # user_answer_rawが空リストの場合も考慮
    user_answer_sorted_tuple = tuple(sorted(user_answer_raw)) if user_answer_raw else ()

    # 正解リスト内の各タプルもソート（比較用）
    correct_answers_sorted = [tuple(sorted(ans)) for ans in correct_answers]

    # 正誤判定
    is_correct = user_answer_sorted_tuple in correct_answers_sorted
    correct_status = "○" if is_correct else "×"

    # リストに追加
    df_data.append({
        "問": question,
        "科目": subject,
        "解答": correct_answers,   # 元の解答データを格納
        "正誤": correct_status,
        "回答": user_answer_raw, # 元の回答データを格納
        "理由": reason
    })

# データフレームを作成
df = pd.DataFrame(df_data)

# カラムの順序を指定
df = df[["問", "科目", "解答", "正誤", "回答", "理由"]]

# --- 結果の表示 ---
#df.style.set_properties(**{'text-align': 'left'})

csv_table = df.to_csv(index=False)
with open('yakuzaishi-kokkashiken_2024_result.csv', 'w', encoding='utf-8') as f:
    f.write(csv_table)

In [11]:
# --- 全体の正答率 ---
total_correct = (df['正誤'] == '○').sum()
total_questions = len(df)
overall_accuracy = total_correct / total_questions

print(f"全体の正答率: {overall_accuracy:.2%} ({total_correct}/{total_questions})") # 正答数/問題数を追記

print("-" * 30) # 区切り線

# --- 科目ごとの正答率（正答数/問題数 付き）---
# '正誤'列が'○'であるかどうかを示すブール値の列を作成
df['is_correct'] = (df['正誤'] == '○')

# 科目ごとに集計（正答数、問題数、正答率）
# aggを使って複数の集計を一度に行う
subject_summary = df.groupby('科目').agg(
    correct_count=('is_correct', 'sum'),  # 正答数を計算 ('is_correct'列のTrueの合計)
    total_count=('is_correct', 'size'), # 問題数を計算 (各グループの行数)
    accuracy=('is_correct', 'mean')      # 正答率を計算 ('is_correct'列の平均値)
)

# 結果を整形して表示する関数
def format_subject_accuracy(row):
    # is_correct列がbool型の場合、sum()は整数を返すのでint()は不要な場合もあるが念のため
    correct = int(row['correct_count'])
    total = int(row['total_count'])
    accuracy_percent = f"{row['accuracy']:.2%}" # パーセンテージ形式
    return f"{accuracy_percent}（{correct}/{total}）"

# applyを使って各行（科目）に関数を適用し、整形後の文字列 Series を作成
formatted_accuracy = subject_summary.apply(format_subject_accuracy, axis=1)

print("科目ごとの正答率:")
print(formatted_accuracy)

全体の正答率: 79.42% (274/345)
------------------------------
科目ごとの正答率:
科目
化学    50.00%（10/20）
実務    83.16%（79/95）
法規    70.00%（21/30）
物理    75.00%（15/20）
生物    90.00%（18/20）
病態    85.00%（34/40）
薬剤    82.50%（33/40）
薬理    85.00%（34/40）
衛生    75.00%（30/40）
dtype: object


In [12]:
stopflg

NameError: name 'stopflg' is not defined

In [ ]:
print("--- Gemini Vision APIによる画像処理を開始します ---")


all_api_responses: List[Union[str, Dict[str, Any]]] = []


for i, pdf_file in enumerate(pdf_files):
    doc = fitz.open(pdf_file+'.pdf')

    Num = 0
    PicsPath = []

    # Output directory
    output_dir = pdf_file

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n[ファイル {i+1}/{len(pdf_files)} の処理開始]")
    print(f"PDF converted to images in '{output_dir}' directory.")

    for page_num in range(doc.page_count):
        page = doc[page_num]
        # Render the page to an image (PNG format)
        pix = page.get_pixmap()
        # Generate the output filename with zero-padded page number
        output_filename = os.path.join(output_dir, f"{Num:03d}.png")
        # Save the image
        pix.save(output_filename)
        Num += 1
        PicsPath.append(output_filename)

    print(f"画像 {len(PicsPath)} 個の処理開始")
    print("-" * 30)
    start_time = time.time()

    # API呼び出し関数を実行
    response = call_gemini_vision_api(
        PicsPath,
        PROMPT,
        MODEL_NAME,
        GENERATION_CONFIG
    )
    all_api_responses.append(json.loads(response.text))

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"  ファイル {i+1} の結果が生成されました。")
    print(f"\n  所要時間: {elapsed_time:.4f} 秒")
    if hasattr(response, 'usage_metadata') and response.usage_metadata:
        print(f"  Prompt Token Count: {response.usage_metadata.prompt_token_count}")
        # print(f"Candidates Token Count: {response.usage_metadata.candidates_token_count}")
        print(f"  Total Token Count: {response.usage_metadata.total_token_count}")
    else:
        print("  Token usage metadata not available.")
    print("-" * 30)

# 全てのチャンクの応答を統合・整形
merged_list, error_info = merge_question_lists_indexed(all_api_responses)

print("--- 結合結果 ---")
print("結合されたリスト:")
# 見やすくするためにjson形式で出力
# sort_keys=True を追加すると、辞書内のキーの順序がソートされ、
# 実行ごとに結果が安定しやすくなります（必須ではない）。
# また、最終的なリスト内の要素の順序は辞書のキー（問ID）に依存します。
marged = json.dumps(merged_list, indent=2, ensure_ascii=False, sort_keys=True)

print(marged)

print("\n--- 重複エラー情報 ---")
if error_info:
  print("以下の問が重複していました（最初に現れたものが保持されます）：")
  for error in error_info:
    print(f"- リスト {error['list_index']} に含まれる '{error['question_id']}'")
else:
  print("重複エラーはありませんでした。")

with open(OUTPUT_FILENAME, "w", encoding="utf-8") as f:
    f.write(marged)
print(f"\n最終結果を '{OUTPUT_FILENAME}' に保存しました。")

print("\n--- 全ての処理が完了しました ---")